
# Otimização de Portfólio de Investimentos

## Problema
Determinar a melhor alocação de ativos em um portfólio para maximizar o retorno esperado e minimizar o risco.

## Como a Têmpera Simulada Pode Ser Aplicada

### Definição do Problema
O objetivo é encontrar a combinação ideal de ações que maximize o retorno esperado do portfólio, dado um nível aceitável de risco.

### Sensores
- Coleta de dados históricos de preços das ações
- Volatilidade
- Correlações entre ativos
- Outros indicadores financeiros

### Atuadores
- Decisões de compra e venda de ações com base na alocação otimizada

### Base de Conhecimento
- Dados históricos de mercado
- Modelos de precificação de ativos
- Teorias financeiras como o Modelo de Markowitz para diversificação de portfólio

### Método de Aprendizagem
A têmpera simulada ajusta iterativamente a alocação de ativos, explorando diferentes combinações e aceitando soluções subótimas temporárias para evitar mínimos locais.

### Função do Agente
Maximizar o retorno esperado do portfólio enquanto minimiza o risco, ajustando a alocação de ativos com base nas condições de mercado.

## Exemplo de Implementação
1. **Inicialização**: Comece com uma alocação inicial de ativos.
2. **Perturbação**: Modifique a alocação atual de forma aleatória.
3. **Avaliação**: Calcule o retorno esperado e o risco da nova alocação.
4. **Aceitação**: Aceite a nova alocação com uma probabilidade que depende da diferença de desempenho e da temperatura atual.
5. **Resfriamento**: Gradualmente reduza a temperatura para refinar a busca.



In [ ]:
import numpy as np
import yfinance as yf

In [ ]:
# Função para calcular o retorno esperado do portfólio
def calcular_retorno(portfolio, retornos_esperados):
    return np.dot(portfolio, retornos_esperados)

In [ ]:
# Função para calcular o risco (desvio padrão) do portfólio
def calcular_risco(portfolio, matriz_covariancia):
    return np.sqrt(np.dot(portfolio.T, np.dot(matriz_covariancia, portfolio)))

In [ ]:
# Função de avaliação do portfólio (retorno ajustado pelo risco)
def avaliar_portfolio(portfolio, retornos_esperados, matriz_covariancia, lambda_risco):
    retorno = calcular_retorno(portfolio, retornos_esperados)
    risco = calcular_risco(portfolio, matriz_covariancia)
    return retorno - lambda_risco * risco

In [ ]:
# Função de têmpera simulada para otimização do portfólio
def tempera_simulada(retornos_esperados, matriz_covariancia, lambda_risco, temperatura_inicial, taxa_resfriamento, iteracoes):
    num_acoes = len(retornos_esperados)
    # Inicializar portfólio com alocação aleatória
    portfolio_atual = np.random.dirichlet(np.ones(num_acoes), size=1)[0]
    melhor_portfolio = portfolio_atual.copy()
    melhor_avaliacao = avaliar_portfolio(melhor_portfolio, retornos_esperados, matriz_covariancia, lambda_risco)

    temperatura = temperatura_inicial

    for i in range(iteracoes):
        # Perturbação: modificar a alocação atual de forma aleatória
        novo_portfolio = portfolio_atual + np.random.normal(0, 0.1, num_acoes)
        novo_portfolio = np.clip(novo_portfolio, 0, 1)
        novo_portfolio /= np.sum(novo_portfolio)

        # Avaliar o novo portfólio
        nova_avaliacao = avaliar_portfolio(novo_portfolio, retornos_esperados, matriz_covariancia, lambda_risco)

        # Aceitar a nova alocação com uma probabilidade que depende da diferença de desempenho e da temperatura atual
        if nova_avaliacao > melhor_avaliacao or np.random.rand() < np.exp((nova_avaliacao - melhor_avaliacao) / temperatura):
            portfolio_atual = novo_portfolio
            melhor_avaliacao = nova_avaliacao
            melhor_portfolio = novo_portfolio

        # Resfriamento: reduzir gradualmente a temperatura
        temperatura *= taxa_resfriamento

    return melhor_portfolio

In [ ]:
# Função para obter dados históricos de preços de ações
def obter_dados_acoes(tickers, periodo='1y'):
    dados = yf.download(tickers, period=periodo)['Adj Close']
    display(dados)
    retornos = dados.pct_change().mean() * 252  # Retornos anuais esperados
    matriz_covariancia = dados.pct_change().cov() * 252  # Matriz de covariância anualizada
    display(matriz_covariancia)
    display(retornos)
    return retornos.values, matriz_covariancia.values

In [ ]:
# Função para avaliar a performance do agente de otimização de portfólio
def avaliar_performance(tickers, lambda_risco=0.5, temperatura_inicial=10000, taxa_resfriamento=0.99, iteracoes=10000):
    # Obter dados históricos das ações
    retornos_esperados, matriz_covariancia = obter_dados_acoes(tickers)

    # Otimizar o portfólio usando têmpera simulada
    melhor_portfolio = tempera_simulada(retornos_esperados, matriz_covariancia, lambda_risco, temperatura_inicial, taxa_resfriamento, iteracoes)

    # Calcular retorno e risco do portfólio otimizado
    retorno_otimizado = calcular_retorno(melhor_portfolio, retornos_esperados)
    risco_otimizado = calcular_risco(melhor_portfolio, matriz_covariancia)

    # Calcular retorno e risco de um portfólio igualmente ponderado (para comparação)
    portfolio_igual = np.ones(len(tickers)) / len(tickers)
    retorno_igual = calcular_retorno(portfolio_igual, retornos_esperados)
    risco_igual = calcular_risco(portfolio_igual, matriz_covariancia)

    # Imprimir resultados
    print("Performance do Agente de Otimização de Portfólio:")
    print("Melhor alocação de portfólio encontrada:", melhor_portfolio)
    print("Retorno esperado do portfólio otimizado:", retorno_otimizado)
    print("Risco (desvio padrão) do portfólio otimizado:", risco_otimizado)
    print("Retorno esperado do portfólio igualmente ponderado:", retorno_igual)
    print("Risco (desvio padrão) do portfólio igualmente ponderado:", risco_igual)

In [ ]:
# Exemplo de uso com ações e parâmetros ajustáveis pelo usuário
tickers = ['AAPL', 'MSFT', 'TSLA']  # Símbolos das ações

lambda_risco = 0.1  # Parâmetro de aversão ao risco
temperatura_inicial = 10000  # Temperatura inicial para a têmpera simulada
taxa_resfriamento = 0.99  # Taxa de resfriamento
iteracoes = 10000  # Número de iterações

avaliar_performance(tickers, lambda_risco=lambda_risco, temperatura_inicial=temperatura_inicial, taxa_resfriamento=taxa_resfriamento, iteracoes=iteracoes)

[*********************100%***********************]  3 of 3 completed


Ticker,AAPL,MSFT,TSLA
Date,,,
2023-11-24 00:00:00+00:00,189.043854,374.628571,235.449997
2023-11-27 00:00:00+00:00,188.864731,375.799805,236.080002
2023-11-28 00:00:00+00:00,189.471771,379.859467,246.720001
2023-11-29 00:00:00+00:00,188.446793,376.038025,244.139999
2023-11-30 00:00:00+00:00,189.023956,376.097565,240.080002
...,...,...,...
2024-11-18 00:00:00+00:00,228.020004,414.929474,338.739990
2024-11-19 00:00:00+00:00,228.279999,416.955414,346.000000
2024-11-20 00:00:00+00:00,229.000000,414.659973,342.029999


Ticker,AAPL,MSFT,TSLA
Ticker,,,
AAPL,0.050807,0.020282,0.044901
MSFT,0.020282,0.038094,0.030902
TSLA,0.044901,0.030902,0.376574


,0
Ticker,
AAPL,0.221561
MSFT,0.126705
TSLA,0.588879


Performance do Agente de Otimização de Portfólio:
Melhor alocação de portfólio encontrada: [0. 0. 1.]
Retorno esperado do portfólio otimizado: 0.5888786513898584
Risco (desvio padrão) do portfólio otimizado: 0.6136560718416009
Retorno esperado do portfólio igualmente ponderado: 0.31238151240895995
Risco (desvio padrão) do portfólio igualmente ponderado: 0.270317745984662
